In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util  
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
import numpy as np
import re
import string
import nltk
%matplotlib inline

In [2]:
#Creating a data frame out of the csv file
df = pd.read_csv(r"C:\Users\arnav\OneDrive\Desktop\sentiment analysis.csv")
df.head() #head function is used to show the top 5 rows of dataframe

C:\Users\arnav\AppData\Local\Temp\ipykernel_8512\1753184612.py:2: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\arnav\OneDrive\Desktop\sentiment analysis.csv")


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    34660 non-null  object 
 1   name                  27900 non-null  object 
 2   asins                 34658 non-null  object 
 3   brand                 34660 non-null  object 
 4   categories            34660 non-null  object 
 5   keys                  34660 non-null  object 
 6   manufacturer          34660 non-null  object 
 7   reviews.date          34621 non-null  object 
 8   reviews.dateAdded     24039 non-null  object 
 9   reviews.dateSeen      34660 non-null  object 
 10  reviews.didPurchase   1 non-null      object 
 11  reviews.doRecommend   34066 non-null  object 
 12  reviews.id            1 non-null      float64
 13  reviews.numHelpful    34131 non-null  float64
 14  reviews.rating        34627 non-null  float64
 15  reviews.sourceURLs 

In [4]:
df["reviews.rating"].value_counts().sort_values(ascending = False)

reviews.rating
5.0    23775
4.0     8541
3.0     1499
1.0      410
2.0      402
Name: count, dtype: int64

In [5]:
#Converting Emojis to their Respective Emotions
df["reviews.text"] = df["reviews.text"].replace([r"\:\)",r"\:\-\)", r"\:\-\}",r"\;\-\}",r"\:\-\>",r"\;\-\)"], ["Happy","Happy","Happy","Happy","Happy","Happy"], regex=True)
df["reviews.text"] = df["reviews.text"].replace([r"\:\-\(",r"\:\(",r"\:\-\|",r"\;\-\(",r"\;\-\<",r"\|\-\{"], ["Sad", "Sad", "Sad", "Sad", "Sad", "Sad",], regex=True)
df["reviews.text"] = df["reviews.text"].replace([r"\:\D",r"\:\'\-\)",r"\:\`\-\("], ["laugh", "tear of joy", "tear of sadness"], regex=True)

In [6]:
# Create regex pattern that matches all punctuation
punctuation_pattern = f"[{re.escape(string.punctuation)}]"

# Replace all punctuation with space in one operation
df["reviews.text"] = df["reviews.text"].str.replace(punctuation_pattern, ' ', regex=True)

In [7]:
permanent = df[['reviews.rating', 'reviews.text', 'reviews.title', 'reviews.username']]
#Find and print the number of null values
print(permanent.isnull())


       reviews.rating  reviews.text  reviews.title  reviews.username
0               False         False          False             False
1               False         False          False             False
2               False         False          False             False
3               False         False          False             False
4               False         False          False             False
...               ...           ...            ...               ...
34655           False         False          False             False
34656           False         False          False             False
34657           False         False          False             False
34658           False         False          False             False
34659           False         False          False             False

[34660 rows x 4 columns]


In [8]:
permanent = df[['reviews.rating', 'reviews.text', 'reviews.title', 'reviews.username']]
#Find and print the number of null values
print(permanent.isnull().sum())
permanent.head()

reviews.rating      33
reviews.text         1
reviews.title        6
reviews.username     7
dtype: int64


,reviews.rating,reviews.text,reviews.title,reviews.username
0,5.0,This product so far has not disappointed My c...,Kindle,Adapter
1,5.0,great for beginner or experienced person Boug...,very fast,truman
2,5.0,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,DaveZ
3,4.0,I ve had my Fire HD 8 two weeks now and I love...,Good!!!,Shacks
4,5.0,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,explore42


In [9]:
#Filtering the null values
check = permanent[permanent['reviews.rating'].isnull()]
print(check[['reviews.text', 'reviews.rating']])

                                            reviews.text  reviews.rating
2886   The Kindle is my first e ink reader  I own an ...             NaN
2887   I m a first time Kindle owner  so I have nothi...             NaN
2888   UPDATE NOVEMBER 2011laughy review is now over ...             NaN
2889   I m a first time Kindle owner  so I have nothi...             NaN
2890   I woke up to a nice surprise this morninglaugh...             NaN
2891   The Kindle is my first e ink reader  I own an ...             NaN
2892   UPDATE NOVEMBER 2011laughr  br  My review is n...             NaN
2893   I woke up to a nice surprise this morninglaugh...             NaN
2894   I use to hate to read but now that I have my K...             NaN
2895   All of them quit working  There s absolutely n...             NaN
2896   I recently ordered my 8th Kindle  It was to re...             NaN
2897                                     Great  Love it              NaN
2898   I researched the purchase of a Kindle for a 

In [10]:
senti= permanent[permanent["reviews.rating"].notnull()]
print(senti[['reviews.text', 'reviews.rating']])

                                            reviews.text  reviews.rating
0      This product so far has not disappointed  My c...             5.0
1      great for beginner or experienced person  Boug...             5.0
2      Inexpensive tablet for him to use and learn on...             5.0
3      I ve had my Fire HD 8 two weeks now and I love...             4.0
4      I bought this for my grand daughter when she c...             5.0
...                                                  ...             ...
34655  This is not appreciably faster than any other ...             3.0
34656  Amazon should include this charger with the Ki...             1.0
34657  Love my Kindle Fire but I am really disappoint...             1.0
34658  I was surprised to find it did not come with a...             1.0
34659  to spite the fact that i have nothing but good...             1.0

[34627 rows x 2 columns]


In [11]:
senti.count()

reviews.rating      34627
reviews.text        34626
reviews.title       34621
reviews.username    34620
dtype: int64

In [12]:
senti.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34627 entries, 0 to 34659
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   reviews.rating    34627 non-null  float64
 1   reviews.text      34626 non-null  object 
 2   reviews.title     34621 non-null  object 
 3   reviews.username  34620 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.3+ MB


In [13]:
nltk.download('stopwords')  # Download the stopwords corpus first


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False